# DSA 2040A - ETL Extract Phase
## Data Warehousing ETL: Airbnb New York Market Listings
#### Aime Muganga - 232

### Introduction

This notebook covers the **Extraction Phase** of the ETL (Extract, Transform, Load) process for the Airbnb listing dataset obtained from InsideAirbnb. The main objective of this phase is to extract and inspect data from both the main dataset (`raw_data.csv`) and the incremental dataset (`incremental_data.csv`), which represents the most recent month of listings. The tasks performed include data loading, profiling, and quality assessment to identify issues such as missing values, duplicates, and inconsistent data types. Additionally, both datasets are merged to simulate real-world incremental data integration, creating a single, validated dataset ready for transformation and analysis.


In [28]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
warnings.filterwarnings('ignore')

# Display settings for better output visibility
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## Part 1: Data Extraction

#### Load Datasets

In [ ]:
# Load the main raw dataset


raw_data = pd.read_csv('data/raw_data.csv')

print(f"Raw data loaded successfully")
print(f"  - Shape: {raw_data.shape[0]:,} rows × {raw_data.shape[1]} columns")

Raw data loaded successfully
  - Shape: 31,995 rows × 18 columns


In [ ]:
# Load Incremental Data
incremental_data = pd.read_csv('data/incremental_data.csv')

print(f"Incremental data loaded successfully")
print(f"  - Shape: {incremental_data.shape[0]:,} rows × {incremental_data.shape[1]} columns")


Incremental data loaded successfully
  - Shape: 4,116 rows × 18 columns


## Data profiling

#### Raw_data preview

In [7]:
raw_data.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,81739,Loft w/ Terrace @ Box House Hotel,417504,The Box House Hotel,Brooklyn,Greenpoint,40.73777,-73.95366,Private room,428.0,1,2,5/12/2011,0.01,29,60,0,Exempt
1,27883,East Village Sanctuary,120223,Jen,Manhattan,East Village,40.72421,-73.98509,Entire home/apt,NaN,30,25,12/10/2011,0.14,1,27,0,NaN
2,464231,Large Room w/ Private Entrance,1530310,Jacques,Brooklyn,Bedford-Stuyvesant,40.68905,-73.95410,Private room,NaN,30,2,5/29/2012,0.01,1,0,0,NaN
3,456578,"Sunny Art Deco Room w/Full Bed, Steps to Metro!",2270454,Helen,Manhattan,Washington Heights,40.85209,-73.93941,Private room,103.0,30,1,6/1/2012,0.01,1,87,0,NaN
4,505231,"Enjoy a 1 Bedroom to share, NYC",2490915,Catherine,Manhattan,Upper East Side,40.77968,-73.95197,Private room,NaN,30,1,6/22/2012,0.01,1,0,0,NaN


In [8]:
print("RAW DATA - STRUCTURE AND DATA TYPES")
print("="*80)
print(raw_data.info())

RAW DATA - STRUCTURE AND DATA TYPES
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31995 entries, 0 to 31994
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              31995 non-null  int64  
 1   name                            31993 non-null  object 
 2   host_id                         31995 non-null  int64  
 3   host_name                       30832 non-null  object 
 4   neighbourhood_group             31995 non-null  object 
 5   neighbourhood                   31995 non-null  object 
 6   latitude                        31995 non-null  float64
 7   longitude                       31995 non-null  float64
 8   room_type                       31995 non-null  object 
 9   price                           17532 non-null  float64
 10  minimum_nights                  31995 non-null  int64  
 11  number_of_reviews               31995 non-null  int64  
 

In [9]:
print("RAW DATA - STATISTICAL SUMMARY")
print("="*80)
print(raw_data.describe())

RAW DATA - STATISTICAL SUMMARY
                 id       host_id      latitude     longitude         price  \
count  3.199500e+04  3.199500e+04  31995.000000  31995.000000  17532.000000   
mean   4.408853e+17  1.649246e+08     40.729335    -73.948323    756.689140   
std    5.408879e+17  1.875140e+08      0.056025      0.053684   4827.268886   
min    2.595000e+03  2.438000e+03     40.500366    -74.251907     10.000000   
25%    2.027918e+07  1.696327e+07     40.689350    -73.983295     82.000000   
50%    4.856267e+07  7.529500e+07     40.726780    -73.955510    150.000000   
75%    9.253078e+17  2.810455e+08     40.762800    -73.929565    279.000000   
max    1.521848e+18  7.214431e+08     40.911390    -73.711822  50104.000000   

       minimum_nights  number_of_reviews  reviews_per_month  \
count    31995.000000       31995.000000       20807.000000   
mean        31.045288          18.681231           0.436760   
std         29.652738          46.099803           0.703548   
min  

#### Incremental_data Preview

In [14]:
incremental_data.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,77765,Superior @ Box House,417504,The Box House Hotel,Brooklyn,Greenpoint,40.737770,-73.953660,Hotel room,NaN,1,71,9/1/2025,0.40,29,58,3,Exempt
1,294263,Loft Suite,417504,The Box House Hotel,Brooklyn,Greenpoint,40.737780,-73.953660,Entire home/apt,428.0,1,162,9/1/2025,1.01,29,57,3,Exempt
2,302705,Wonderful Room on the UES 2,1439222,Virginia,Manhattan,Upper East Side,40.772980,-73.958130,Private room,75.0,30,9,9/1/2025,0.13,1,312,3,NaN
3,765569,"Boutique Brooklyn Condo, Quiet St, Great Location",4036700,Anna,Brooklyn,Downtown Brooklyn,40.696190,-73.983240,Entire home/apt,170.0,30,89,9/1/2025,0.57,1,84,7,NaN
4,4550602,KLO Room #3 With Private Bathroom & Private Deck,23533897,Fatou,Brooklyn,Crown Heights,40.675003,-73.951668,Private room,178.0,1,139,9/1/2025,1.06,6,323,22,Exempt


In [15]:
print("INCREMENTAL DATA - STRUCTURE")
print("="*80)
print(incremental_data.info())

INCREMENTAL DATA - STRUCTURE
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4116 entries, 0 to 4115
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              4116 non-null   int64  
 1   name                            4116 non-null   object 
 2   host_id                         4116 non-null   int64  
 3   host_name                       4113 non-null   object 
 4   neighbourhood_group             4116 non-null   object 
 5   neighbourhood                   4116 non-null   object 
 6   latitude                        4116 non-null   float64
 7   longitude                       4116 non-null   float64
 8   room_type                       4116 non-null   object 
 9   price                           3796 non-null   float64
 10  minimum_nights                  4116 non-null   int64  
 11  number_of_reviews               4116 non-null   int64  
 12  last_

In [16]:
print("Incremental DATA - STATISTICAL SUMMARY")
print("="*80)
print(incremental_data.describe())

Incremental DATA - STATISTICAL SUMMARY
                 id       host_id     latitude    longitude         price  \
count  4.116000e+03  4.116000e+03  4116.000000  4116.000000   3796.000000   
mean   6.681655e+17  2.587543e+08    40.719847   -73.939789    328.767650   
std    5.336843e+17  2.258311e+08     0.057447     0.063780   2239.899586   
min    7.097000e+03  1.678000e+03    40.520340   -74.231730     13.000000   
25%    4.450310e+07  3.549167e+07    40.679462   -73.985190    110.000000   
50%    8.180000e+17  2.067676e+08    40.718160   -73.949769    168.000000   
75%    1.120000e+18  4.726034e+08    40.757323   -73.916543    278.000000   
max    1.510000e+18  7.144086e+08    40.912017   -73.726060  50052.000000   

       minimum_nights  number_of_reviews  reviews_per_month  \
count     4116.000000         4116.00000        4116.000000   
mean         9.680515           94.48275           2.764468   
std         14.966279          147.52329           3.899811   
min          1.

## Data Quality Assessment

### Issue 1: Missing Values Analysis

In [18]:
## Issue 1: Missing Values Analysis
print("\n### ISSUE 1: MISSING VALUES ###\n")

# Check missing values in raw data
missing_raw = raw_data.isnull().sum()
missing_raw_pct = (raw_data.isnull().sum() / len(raw_data)) * 100

missing_summary_raw = pd.DataFrame({
    'Column': missing_raw.index,
    'Missing_Count': missing_raw.values,
    'Missing_Percentage': missing_raw_pct.values
}).sort_values('Missing_Count', ascending=False)

print("RAW DATA - Missing Values:")
print(missing_summary_raw[missing_summary_raw['Missing_Count'] > 0])

print(f"\n Found {(missing_raw > 0).sum()} columns with missing values in raw data")

# Check missing values in incremental data
missing_inc = incremental_data.isnull().sum()
missing_inc_pct = (incremental_data.isnull().sum() / len(incremental_data)) * 100

missing_summary_inc = pd.DataFrame({
    'Column': missing_inc.index,
    'Missing_Count': missing_inc.values,
    'Missing_Percentage': missing_inc_pct.values
}).sort_values('Missing_Count', ascending=False)

print("\nINCREMENTAL DATA - Missing Values:")
print(missing_summary_inc[missing_summary_inc['Missing_Count'] > 0])

print(f"\n Found {(missing_inc > 0).sum()} columns with missing values in incremental data")



### ISSUE 1: MISSING VALUES ###

RAW DATA - Missing Values:
               Column  Missing_Count  Missing_Percentage
17            license          29703           92.836381
9               price          14463           45.203938
12        last_review          11188           34.967964
13  reviews_per_month          11188           34.967964
3           host_name           1163            3.634943
1                name              2            0.006251

 Found 6 columns with missing values in raw data

INCREMENTAL DATA - Missing Values:
       Column  Missing_Count  Missing_Percentage
17    license           1032           25.072886
9       price            320            7.774538
3   host_name              3            0.072886

 Found 3 columns with missing values in incremental data


Missing values are significantly present in this dataset, affecting 6 columns in raw data 
and 3 columns in incremental data. The severity varies considerably:

**Critical Missing Data:**
- LICENSE (92.84% in raw, 25.07% in incremental): Nearly all records lack license info
  → This suggests license data is poorly maintained or not required in many areas
  
- PRICE (45.20% in raw, 7.77% in incremental): Almost half of raw listings missing price
  → Could indicate listings that are temporarily unavailable or improperly scraped
  → Incremental data shows improvement (only 7.77% missing)

**Moderate Missing Data:**
- LAST_REVIEW (34.97% in raw): Over one-third of listings have never been reviewed
  → Expected for new listings or unpopular properties
  
- REVIEWS_PER_MONTH (34.97% in raw): Missing at same rate as last_review
  → These are dependent - no reviews means no review rate

**Minor Missing Data:**
- HOST_NAME (3.63% in raw, 0.07% in incremental): Minimal impact
- NAME (0.006% in raw): Negligible, likely data entry errors

**Strategy for Handling:**
Different strategies will be applied based on variable type and business context:
- Numerical (price): Imputation using median (robust to outliers)
- Dates (last_review): Keep as null or impute with a flag value
- Categorical (license): Drop column due to 92% missing rate
- Derived fields (reviews_per_month): Fill with 0 for never-reviewed listings

### Issue 2: Duplicate Records

In [20]:
### Issue 2: Duplicate Records
print("\n" + "-"*80)
print("### ISSUE 2: DUPLICATE RECORDS ###\n")

# Check for duplicates based on ID (primary key)
duplicates_raw_id = raw_data.duplicated(subset=['id'], keep=False).sum()
duplicates_inc_id = incremental_data.duplicated(subset=['id'], keep=False).sum()

print(f"RAW DATA:")
print(f"  - Duplicate IDs: {duplicates_raw_id}")
print(f"  - Completely duplicate rows: {raw_data.duplicated().sum()}")

print(f"\nINCREMENTAL DATA:")
print(f"  - Duplicate IDs: {duplicates_inc_id}")
print(f"  - Completely duplicate rows: {incremental_data.duplicated().sum()}")

# Check for overlapping IDs between datasets
overlapping_ids = set(raw_data['id']).intersection(set(incremental_data['id']))
print(f"\nOVERLAPPING IDs between datasets: {len(overlapping_ids)}")




--------------------------------------------------------------------------------
### ISSUE 2: DUPLICATE RECORDS ###

RAW DATA:
  - Duplicate IDs: 0
  - Completely duplicate rows: 0

INCREMENTAL DATA:
  - Duplicate IDs: 2526
  - Completely duplicate rows: 0

OVERLAPPING IDs between datasets: 0


The duplicate analysis reveals an interesting pattern:

**Raw Data:** Clean - No duplicate IDs or rows
  → Indicates the original dataset was well-maintained

**Incremental Data:** 2,526 duplicate IDs (61.4% of 4,116 records!)
  → This is unusual and suggests the incremental file may contain:
     * Multiple snapshots of the same listings over time
     * Data collection methodology that captures updates multiple times
     * Possible data quality issue in the source extraction

**No Overlapping IDs between datasets:**
  → This means raw_data and incremental_data contain completely different listings
  → The incremental data represents genuinely NEW listings, not updates to existing ones
  → Our merge strategy should be a simple append, not an update operation

**Implication for Merge:**
Since there are no overlapping IDs, we will:
1. Remove duplicates within incremental_data first (keep latest occurrence)
2. Then append to raw_data
3. This gives us the complete universe of unique listings

### Issue 3: Data Type Inconsistencies

In [22]:
### Issue 3: Data Type Inconsistencies
print("\n" + "-"*80)
print("### ISSUE 3: DATA TYPE INCONSISTENCIES ###\n")

print("RAW DATA - Data Types:")
print(raw_data.dtypes)


# Sample price values to check for currency symbols
print("\nSample 'price' values:")
print(raw_data['price'].head(10))


--------------------------------------------------------------------------------
### ISSUE 3: DATA TYPE INCONSISTENCIES ###

RAW DATA - Data Types:
id                                  int64
name                               object
host_id                             int64
host_name                          object
neighbourhood_group                object
neighbourhood                      object
latitude                          float64
longitude                         float64
room_type                          object
price                             float64
minimum_nights                      int64
number_of_reviews                   int64
last_review                        object
reviews_per_month                 float64
calculated_host_listings_count      int64
availability_365                    int64
number_of_reviews_ltm               int64
license                            object
dtype: object

Sample 'price' values:
0    428.0
1      NaN
2      NaN
3    103.0
4      NaN
5 



Data type analysis reveals several inconsistencies that need correction:

**Primary Issue - Date Format:**
- LAST_REVIEW is stored as 'object' (string) instead of datetime
  → Cannot perform time-based analysis (e.g., how recent are reviews?)
  → Cannot calculate days since last review or filter by date ranges
  → Must convert to datetime64 format for temporal analysis

**Price Data - Already Numeric:**
- PRICE is correctly stored as float64 (not string with currency symbols)
  → Good news: No need to strip '$' or ',' characters
  → Sample shows mixture of values and NaN (as expected from Issue 1)

**ID Fields - Consider Categorical:**
- ID and HOST_ID are stored as int64
  → While numeric, they're identifiers (not quantities to calculate)
  → Could convert to string/category type to prevent accidental math operations
  → However, int64 is acceptable for indexing purposes

**All Other Fields - Appropriate Types:**
- Numerical fields (latitude, longitude, minimum_nights, etc.): Correct types
- Text fields (name, host_name, room_type, etc.): Correctly as 'object'

**Action Required:**
Convert 'last_review' from object → datetime64 during transformation phase

In [23]:
### Issue 4: Data Range and Outliers
print("\n" + "-"*80)
print("### ISSUE 4: DATA RANGE ANOMALIES & OUTLIERS ###\n")

# Check for unrealistic values
print("PRICE Analysis:")
print(f"  - Min price: ${raw_data['price'].min()}")
print(f"  - Max price: ${raw_data['price'].max()}")
print(f"  - Prices = 0: {(raw_data['price'] == 0).sum()}")
print(f"  - Prices > $10,000: {(raw_data['price'] > 10000).sum()}")

print("\nMINIMUM NIGHTS Analysis:")
print(f"  - Min: {raw_data['minimum_nights'].min()}")
print(f"  - Max: {raw_data['minimum_nights'].max()}")
print(f"  - Values > 365: {(raw_data['minimum_nights'] > 365).sum()}")

print("\nAVAILABILITY Analysis:")
print(f"  - Min: {raw_data['availability_365'].min()}")
print(f"  - Max: {raw_data['availability_365'].max()}")

print("\n📊 OBSERVATION:")
print("   - Zero or extremely high prices may need investigation")
print("   - Minimum nights > 365 days seems unusual")
print("   - These outliers may be valid but should be flagged for review")


--------------------------------------------------------------------------------
### ISSUE 4: DATA RANGE ANOMALIES & OUTLIERS ###

PRICE Analysis:
  - Min price: $10.0
  - Max price: $50104.0
  - Prices = 0: 0
  - Prices > $10,000: 210

MINIMUM NIGHTS Analysis:
  - Min: 1
  - Max: 1124
  - Values > 365: 25

AVAILABILITY Analysis:
  - Min: 0
  - Max: 365

📊 OBSERVATION:
   - Zero or extremely high prices may need investigation
   - Minimum nights > 365 days seems unusual
   - These outliers may be valid but should be flagged for review




Significant outliers and unusual values detected across key variables:

**PRICE Anomalies:**
- Range: $10 to $50,104 per night
- 210 listings (1.2%) priced above $10,000
  → Could be luxury properties, entire buildings, or data errors
  → Max price of $50,104 seems unrealistic even for luxury NYC properties
  → These extreme values will skew analysis and should be investigated
  
**MINIMUM_NIGHTS Anomalies:**
- Range: 1 to 1,124 nights (over 3 years!)
- 25 listings require minimum stays > 365 days
  → Likely long-term rental properties or corporate housing
  → May not be suitable for typical Airbnb analysis (vacation rentals)
  → Could indicate mis-categorized housing or data entry errors

**AVAILABILITY Patterns:**
- Range: 0 to 365 days (expected and valid)
  → 0 = fully booked or inactive
  → 365 = available year-round
  → This field appears clean with no anomalies

**Business Impact:**
These outliers can significantly distort:
- Average price calculations (mean pulled up by extreme values)
- Market analysis (mixing vacation rentals with long-term housing)
- Recommendation systems (suggesting $50k/night properties)

**Handling Strategy:**
- Define reasonable business rules (e.g., price < $5,000, min_nights < 90)
- Flag outliers for review rather than automatic deletion
- Use median instead of mean for central tendency calculations

### Merging raw_data and increment_data into one dataset

In [26]:
# 5. Merge Datasets

print("\n" + "="*80)
print("MERGING DATASETS")
print("="*80)

print(f"\nBefore Merge:")
print(f"  - Raw data: {len(raw_data):,} rows")
print(f"  - Incremental data: {len(incremental_data):,} rows")
print(f"  - Overlapping IDs: {len(overlapping_ids)}")

# Strategy: Combine datasets, keeping the most recent version for overlapping IDs
# Since incremental represents newer data, we'll prioritize it for duplicate IDs

# Merge approach: Concatenate and drop duplicates, keeping last (incremental)
merged_data = pd.concat([raw_data, incremental_data], ignore_index=True)
print(f"\nAfter Concatenation: {len(merged_data):,} rows")

# Remove duplicates based on ID, keeping the last occurrence (from incremental)
merged_data = merged_data.drop_duplicates(subset=['id'], keep='last')
print(f"After Removing Duplicate IDs: {len(merged_data):,} rows")

print("\n MERGE EXPLANATION:")
print("   WHY: Incremental data contains updated/new listings that should")
print("        be incorporated into our main dataset for complete analysis")
print("   HOW: Concatenated both datasets and removed duplicate IDs,")
print("        keeping the last occurrence (from incremental) as it's more recent")
print(f"   RESULT: Final dataset has {len(merged_data):,} unique listings")

# Verify merge
print(f"\nMerge Verification:")
print(f"  - Expected max: {len(raw_data) + len(incremental_data):,}")
print(f"  - Actual result: {len(merged_data):,}")
print(f"  - Duplicates removed: {len(raw_data) + len(incremental_data) - len(merged_data):,}")



MERGING DATASETS

Before Merge:
  - Raw data: 31,995 rows
  - Incremental data: 4,116 rows
  - Overlapping IDs: 0

After Concatenation: 36,111 rows
After Removing Duplicate IDs: 33,928 rows

 MERGE EXPLANATION:
   WHY: Incremental data contains updated/new listings that should
        be incorporated into our main dataset for complete analysis
   HOW: Concatenated both datasets and removed duplicate IDs,
        keeping the last occurrence (from incremental) as it's more recent
   RESULT: Final dataset has 33,928 unique listings

Merge Verification:
  - Expected max: 36,111
  - Actual result: 33,928
  - Duplicates removed: 2,183


### **Why Merge is Necessary**

1. **Complete Dataset Creation:**  
   The raw data containing 31,995 listings represents the historical baseline of Airbnb listings, while the incremental data with 4,116 listings corresponds to the most recent month and includes newly added properties. Without merging these two datasets, the analysis would be incomplete and miss recent additions to the market. By combining both datasets, we obtain a more comprehensive view of the Airbnb market that includes both older and newer listings, ensuring that our analysis reflects the most current state of the data.

2. **Real-World ETL Pattern:**  
   In real production systems, new data usually arrives in periodic batches rather than as a full dataset each time. Merging the incremental data demonstrates an understanding of incremental data loading, which is more efficient than re-downloading or replacing the entire dataset whenever updates occur. This approach mirrors how actual data warehouses function in practice, where updates happen through daily, weekly, or monthly refreshes to keep the system up to date.

3. **Business Intelligence Value:**  
   Performing the merge ensures complete market coverage by consolidating all active listings into one dataset for accurate analysis. It also provides temporal completeness by capturing both long-standing and newly added properties. With the merged dataset, we can perform meaningful trend analyses, such as comparing older listings with newer ones to study market growth, seasonal trends, or shifts in property availability over time.


### Save Validated Data

In [25]:
## 6. Save Validated Data

print("\n" + "="*80)
print("SAVING VALIDATED DATA")
print("="*80)

# Create data directory if it doesn't exist
os.makedirs('data', exist_ok=True)

# Save the merged dataset as validated data
output_path = 'data/validated_data.csv'
merged_data.to_csv(output_path, index=False)
print(f"✓ Validated merged data saved to: {output_path}")
print(f"  - Rows: {len(merged_data):,}")
print(f"  - Columns: {merged_data.shape[1]}")


SAVING VALIDATED DATA
✓ Validated merged data saved to: data/validated_data.csv
  - Rows: 33,928
  - Columns: 18


### Conclusion

In summary, this extraction phase successfully loaded, explored, and validated both the raw and incremental Airbnb datasets. Key data quality issues such as missing values and inconsistent data types were identified, and appropriate notes were made for cleaning in the transformation stage. By merging the incremental data with the main dataset, a complete and up-to-date Airbnb listings dataset was produced, accurately reflecting current market conditions. The validated merged data was then saved to **`data/validated_data.csv`**, ensuring that the dataset is ready for the next phase—data transformation and enrichment.

